In [20]:
import markdown
from weasyprint import HTML
from pathlib import Path
import sys

In [21]:
def md_to_pdf(md_file: str, pdf_file: str = None):
    """Convert markdown to PDF"""
    
    # Setup paths
    md_path = Path(md_file)
    if not md_path.exists():
        print(f"❌ Error: File not found: {md_file}")
        sys.exit(1)
    
    if pdf_file is None:
        pdf_file = str(md_path.with_suffix('.pdf'))
    
    # Read markdown content
    print(f"📖 Reading {md_file}...")
    md_content = md_path.read_text(encoding='utf-8')
    
    # Convert markdown to HTML
    print("🔄 Converting to HTML...")
    html_content = markdown.markdown(
        md_content,
        extensions=['extra', 'codehilite', 'tables', 'toc', 'fenced_code']
    )
    
    # Create styled HTML document
    styled_html = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="utf-8">
        <style>
            @page {{
                size: A4;
                margin: 2.5cm;
            }}
            body {{
                font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Arial, sans-serif;
                line-height: 1.6;
                color: #333;
            }}
            h1 {{
                color: #2c3e50;
                border-bottom: 3px solid #3498db;
                padding-bottom: 10px;
                margin-top: 20px;
            }}
            h2 {{
                color: #34495e;
                border-bottom: 1px solid #ddd;
                padding-bottom: 5px;
                margin-top: 15px;
            }}
            h3 {{
                color: #555;
                margin-top: 12px;
            }}
            code {{
                background-color: #f4f4f4;
                padding: 2px 6px;
                border-radius: 3px;
                font-family: 'Courier New', monospace;
                font-size: 0.9em;
            }}
            pre {{
                background-color: #f4f4f4;
                padding: 15px;
                border-radius: 5px;
                overflow-x: auto;
                border-left: 4px solid #3498db;
            }}
            pre code {{
                background: none;
                padding: 0;
            }}
            blockquote {{
                border-left: 4px solid #ddd;
                padding-left: 15px;
                color: #666;
                margin: 15px 0;
                font-style: italic;
            }}
            table {{
                border-collapse: collapse;
                width: 100%;
                margin: 15px 0;
            }}
            th, td {{
                border: 1px solid #ddd;
                padding: 10px;
                text-align: left;
            }}
            th {{
                background-color: #f4f4f4;
                font-weight: bold;
            }}
            tr:nth-child(even) {{
                background-color: #f9f9f9;
            }}
            a {{
                color: #3498db;
                text-decoration: none;
            }}
            a:hover {{
                text-decoration: underline;
            }}
            ul, ol {{
                margin: 10px 0;
                padding-left: 30px;
            }}
            li {{
                margin: 5px 0;
            }}
        </style>
    </head>
    <body>
        {html_content}
    </body>
    </html>
    """
    
    # Convert HTML to PDF
    print(f"📄 Generating PDF...")
    try:
        HTML(string=styled_html).write_pdf(pdf_file)
        print(f"✅ Success! PDF created: {pdf_file}")
    except Exception as e:
        print(f"❌ Error: {e}")
        sys.exit(1)

In [22]:
md_to_pdf('researcher/output/report.md', 'researcher/output/report_vn.pdf')

Ignored `overflow-x: auto` at 38:17, unknown property.


📖 Reading researcher/output/report.md...
🔄 Converting to HTML...
📄 Generating PDF...


maxp pruned
cmap pruned
fpgm dropped
prep dropped
cvt  dropped
kern dropped
post pruned
GPOS dropped
GSUB dropped
glyf pruned
Added gid0 to subset
Closing glyph list over 'glyf': 126 glyphs before
Glyph names: ['.notdef', 'A', 'B', 'C', 'D', 'Dcroat', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'Uacute', 'V', 'W', 'X', 'a', 'aacute', 'abreve', 'abreveacute', 'abrevedotbelow', 'abrevegrave', 'abrevetilde', 'acircumflex', 'acircumflexacute', 'acircumflexdotbelow', 'acircumflexgrave', 'acircumflexhookabove', 'acircumflextilde', 'adotbelow', 'agrave', 'ahookabove', 'atilde', 'b', 'c', 'colon', 'comma', 'd', 'dcroat', 'e', 'eacute', 'ecircumflex', 'ecircumflexacute', 'ecircumflexdotbelow', 'ecircumflexgrave', 'ecircumflexhookabove', 'ecircumflextilde', 'emdash', 'etilde', 'f', 'five', 'g', 'h', 'hyphen', 'i', 'iacute', 'idotbelow', 'igrave', 'ihookabove', 'itilde', 'k', 'l', 'm', 'n', 'o', 'oacute', 'ocircumflex', 'ocircumflexacute', 'ocircumflexdotbelow'

✅ Success! PDF created: researcher/output/report_vn.pdf


In [ ]:
from crewai import Agent, Crew, Process, Task
from crewai.project import CrewBase, agent, crew, task
from crewai.agents.agent_builder.base_agent import BaseAgent
from typing import List

@CrewBase
class EngineeringTeam():
    """EngineeringTeam crew"""

    agents: List[BaseAgent]
    tasks: List[Task]

    @agent
    def product_manager(self) -> Agent:
        return Agent(
            config=self.agents_config['product_manager'], # type: ignore[index]
            max_execution_time=300, 
            max_retry_limit=3,
            verbose=True
        )

    @agent
    def engineering_lead(self) -> Agent:
        return Agent(
            config=self.agents_config['engineering_lead'], # type: ignore[index]
            max_execution_time=300, 
            max_retry_limit=3,
            verbose=True
        )

    @agent
    def backend_engineer(self) -> Agent:
        return Agent(
            config=self.agents_config['backend_engineer'], # type: ignore[index]
            allow_code_execution=True,
            code_execution_mode="safe", 
            max_execution_time=500, 
            max_retry_limit=3,
            verbose=True
        )    

    @task
    def create_userstories_task(self) -> Task:
        return Task(
            config=self.tasks_config['create_userstories_task'] # type: ignore[index]
        )

    @task
    def design_task(self) -> Task:
        return Task(
            config=self.tasks_config['design_task'] # type: ignore[index]
        )

    @task
    def code_task(self) -> Task:
        return Task(
            config=self.tasks_config['code_task'] # type: ignore[index]
        )

    @crew
    def crew(self) -> Crew:
        """Creates the EngineeringTeam crew"""
       
        return Crew(
            agents=self.agents, # Automatically created by the @agent decorator
            tasks=self.tasks, # Automatically created by the @task decorator
            process=Process.sequential,
            verbose=True,
            # process=Process.hierarchical, # In case you wanna use that instead https://docs.crewai.com/how-to/Hierarchical/
        )
